## This notebook has the goal of making the Mixture of Factor Analyzers (MFA) model, along with benchmark models

The models are to be saved to a models-folder in the project

In [ ]:
from hypso import Hypso